In [2]:
# Loading google palm and testing 
from langchain.llms import GooglePalm

api_key = 'you api key'

llm = GooglePalm(google_api_key=api_key, temperature=0.1)

In [ ]:
test = llm('Hi google palm')
print(test)

In [5]:
# loading the csv data
from langchain.document_loaders.csv_loader import CSVLoader

loader = CSVLoader(file_path='faqs.csv', source_column="prompt",encoding="latin1")

data = loader.load()

In [ ]:
print(data[0].page_content)

In [11]:
#Hugging face embeddings

from langchain.embeddings import HuggingFaceInstructEmbeddings
instructor_embeddings = HuggingFaceInstructEmbeddings(model_name="hkunlp/instructor-large")


load INSTRUCTOR_Transformer
max_seq_length  512


In [12]:
# creating a vector db using facebook ai similarity search (will take some time)
from langchain.vectorstores import FAISS
vectorDb = FAISS.from_documents(documents=data, embedding=instructor_embeddings)
retriever = vectorDb.as_retriever(score_threshold = 0.7)

In [13]:
rdocs = retriever.get_relevant_documents("how about job placement support?")
rdocs

[Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you provide any job assistance?', 'row': 11}),
 Document(page_content='prompt: Will this course guarantee me a job?\nresponse: We created a much lighter version of this course on YouTube available for free (click this link) and many people gave us feedback that they were able to fetch jobs (see testimonials). Now this paid course is at least 5x better than the YouTube course which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.', metadata={'source': 'Will this 

In [14]:

from langchain.prompts import PromptTemplate

prompt_template = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""


PROMPT = PromptTemplate(
    template=prompt_template, input_variables=["context", "question"]
)
chain_type_kwargs = {"prompt": PROMPT}


from langchain.chains import RetrievalQA

chain = RetrievalQA.from_chain_type(llm=llm,
                            chain_type="stuff",
                            retriever=retriever,
                            input_key="query",
                            return_source_documents=True,
                            chain_type_kwargs=chain_type_kwargs)


In [15]:
chain('Do you provide job assistance and also do you provide job gurantee?')

{'query': 'Do you provide job assistance and also do you provide job gurantee?',
 'result': 'Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters. The courses included in this bootcamp are done by 9000+ learners and many of them have secured a job which gives us ample confidence that you will be able to get a job. However, we want to be honest and do not want to make any impractical promises! Our guarantee is to prepare you for the job market by teaching the most relevant skills, knowledge & timeless principles good enough to fetch the job.',
 'source_documents': [Document(page_content='prompt: Do you provide any job assistance?\nresponse: Yes, We help you with resume and interview preparation along with that we help you in building online credibility, and based on requirements we refer candidates to potential recruiters.', metadata={'source': 'Do you prov

In [16]:
chain("What is the price of your machine learning course?")

{'query': 'What is the price of your machine learning course?',
 'result': "I don't know.",
 'source_documents': [Document(page_content='prompt: Iï¿½m not sure if this course is good enough for me to invest some money. What can I do?\nresponse: Donï¿½t worry. Many videos in this course are free so watch them to get an idea of the quality of teaching. Dhaval Patel (the course instructor) runs a popular data science YouTube channel called Codebasics. On that, you can watch his videos and read comments to get an idea of his teaching style', metadata={'source': 'Iï¿½m not sure if this course is good enough for me to invest some money. What can I do?', 'row': 20}),
  Document(page_content='prompt: Will the course be upgraded when there are new features in Power BI?\nresponse: Yes, the course will be upgraded periodically based on the new features in Power BI, and learners who have already bought this course will have free access to the upgrades.', metadata={'source': 'Will the course be upg